<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/model_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analysis of model results
Edit `metric_dict` to change weights for various metrics. Scroll down to see analysis of final round of experiments (with 20-30 epochs.)



In [0]:
# Input metric weights from 0 to 1.
metric_dict = {'delta_gender':.9,
               'delta_party':.6,
               'final_gender':.7,
               'final_party':.7,
               'best_gender':.8,
               'best_party':.7,
               'percent_W':.9,
               'percent_D':.8
               }

In [1]:
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [2]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [0]:
from google.colab import files
import io 

Don't need to do the next few cells unless uploading new results from local machine.

In [0]:
# Get manually inputted model results. [11/23/19 3pm]

uploaded = files.upload()
model_results = pd.read_csv(io.BytesIO(uploaded['model_results.csv']))

Saving model_results.csv to model_results (6).csv


In [0]:
# Download the new dataframe to local machine.

model_results.to_csv('model_results.csv', header=True, index=False)
files.download('model_results.csv')

In [0]:
  !gsutil cp model_results\ \(6\).csv gs://fb-congressional-data/models/model_results.csv

Copying file://model_results (6).csv [Content-Type=text/csv]...
/ [1 files][ 21.9 KiB/ 21.9 KiB]                                                
Operation completed over 1 objects/21.9 KiB.                                     


Start here to get results from the bucket.

In [0]:
model_results = pd.read_csv("gs://fb-congressional-data/models/model_results.csv")

In [0]:
model_results.head()

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name
0,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,10,5000,2,32,2,0.2,10,50,1,0.5206,0.7662,0.8490,0.4936,0.7781,0.8489,0.8490,1,-0.0001,0.0000,1.559,NaN,test3_model_0_gender_1574450353.2073507.h5
1,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,20,5000,2,32,2,0.2,10,50,2,0.5180,0.7663,0.8501,0.4891,0.7789,0.8472,0.8486,5,-0.0029,-0.0015,1.861,NaN,test3_model_2_gender_1574450353.2073507.h5
2,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,30,5000,2,32,2,0.2,10,50,3,0.5106,0.7699,0.8511,0.4844,0.7813,0.8496,0.8517,3,-0.0015,0.0006,1.994,NaN,LPFriday11_22_model_1_gender_1574459044.4948301.h5
3,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,32,2,0.2,10,50,4,0.5143,0.7677,0.8452,0.4830,0.7820,0.8466,0.8477,4,0.0014,0.0025,2.384,NaN,LPFriday11_22_model_3_gender_1574459044.4948301.h5
4,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,100,5000,2,32,2,0.2,10,50,5,0.5068,0.7718,0.8467,0.4813,0.7829,0.8473,0.8490,3,0.0006,0.0023,2.204,NaN,LPFriday11_22_model_5_gender_1574459044.4948301.h5


In [0]:
def get_model_metric(results_df, metric_dict,n):
  '''Take a dataframe of model results, metrics and number of top models
  and return that dataframe with a new model_metric column. The dataframe must have "saved_model_name",
  "delta_val_acc","final_val_acc","best_val_acc", "percent_W", and "percent_D"'''

  gender_results = results_df[results_df['saved_model_name'].str.contains("gender")]
  party_results = results_df[results_df['saved_model_name'].str.contains("party")]

  gender_best_delta = gender_results.sort_values('delta_val_acc', ascending=False).head(n)
  gender_best_delta['model_metric'] = metric_dict['delta_gender']
  gender_best_final = gender_results.sort_values("final_val_acc", ascending=False).head(n)
  gender_best_final['model_metric'] = metric_dict['final_gender']
  gender_best_acc = gender_results.sort_values("best_val_acc", ascending=False).head(n)
  gender_best_acc['model_metric'] = metric_dict['best_gender']
  gender_best_W = gender_results.sort_values("percent_W", ascending=False).head(n)
  gender_best_W['model_metric'] = metric_dict['percent_W']

  party_best_delta = party_results.sort_values('delta_val_acc', ascending=False).head(n)
  party_best_delta['model_metric'] = metric_dict['delta_party']
  party_best_final = party_results.sort_values("final_val_acc", ascending=False).head(n)
  party_best_final['model_metric'] = metric_dict['final_party']
  party_best_acc = party_results.sort_values("best_val_acc", ascending=False).head(n)
  party_best_acc['model_metric'] = metric_dict['best_party']
  party_best_D = party_results.sort_values("percent_D", ascending=False).head(n)
  party_best_D['model_metric'] = metric_dict['percent_D']

  metrics_df = gender_best_delta.append([gender_best_final,gender_best_acc,gender_best_W,party_best_delta,
                                              party_best_delta,party_best_final,party_best_acc,party_best_D])
  best_indices = metrics_df.groupby('model_idx')['model_metric'].sum().reset_index()
  best_results_df = results_df.merge(best_indices,on='model_idx')
  return best_results_df

In [0]:
best_models = get_model_metric(model_results,metric_dict,10).sort_values('model_metric', ascending=False)
best_models.head(20)

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name,model_metric
19,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5901,0.6750,0.7351,0.5482,0.7031,0.7504,0.7505,7,0.0153,0.0154,NaN,7.925,LPSaturday11_23_model_2_party_1574532795.1390407.h5,3.8
18,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5067,0.7710,0.8483,0.4793,0.7833,0.8510,0.8512,9,0.0027,0.0029,1.1720,NaN,LPSaturday11_23_model_2_gender_1574532795.1390407.h5,3.8
21,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.8,10,50,18,0.6449,0.6367,0.7388,0.5807,0.6817,0.7410,0.7411,5,0.0022,0.0023,NaN,31.855,truncated to 6 epochs; met output size limit; party model,3.2
20,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.8,10,50,18,0.5344,0.7583,0.8448,0.4968,0.7764,0.8516,0.8516,10,0.0068,0.0068,0.7170,NaN,LPSaturday11_23_model_3_gender_1574532795.1390407.h5,3.2
14,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.0,10,50,15,0.4843,0.7809,0.8502,0.4597,0.7899,0.8408,0.8502,1,-0.0094,0.0000,3.9940,NaN,LPSaturday11_23_model_0_gender_1574532795.1390407.h5,2.9
15,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.0,10,50,15,0.5535,0.7008,0.7103,0.5125,0.7269,0.7416,0.7416,10,0.0313,0.0313,NaN,15.107,LPSaturday11_23_model_0_party_1574532795.1390407.h5,2.9
74,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,1,128,2,0.5,10,50,49,0.5189,0.7658,0.8480,0.4828,0.7823,0.8488,0.8498,2,0.0008,0.0018,1.3660,NaN,emSaturdayRound1_model_4_gender_1574526502.7369754.h5,2.6
25,Y,2,0,0,1813217.0,1108843.0,0,0,1698252,594655,25,10,1000,50,5000,2,32,2,0.2,10,50,20,0.5846,0.6780,0.7192,0.5363,0.7096,0.7531,0.7531,10,0.0339,0.0339,NaN,12.976,LPFriday11_22_model_1_party_1574533565.182682.h5,2.6
24,Y,2,2168175,765916,NaN,NaN,1948371,344536,0,0,25,10,1000,50,5000,2,32,2,0.2,10,50,20,0.4952,0.7785,0.8487,0.4667,0.7902,0.8475,0.8494,9,-0.0012,0.0007,2.7990,NaN,LPFriday11_22_model_1_gender_1574533565.182682.h5,2.6
75,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,1,128,2,0.5,10,50,49,0.5997,0.6701,0.7215,0.5539,0.7008,0.7479,0.7515,9,0.0264,0.0300,NaN,13.997,emSaturdayRound1_model_4_party_1574526502.7369754.h5,2.6


## Analysis of results from final 10 model experiments.

Only run the next few cells to get new results from local machine.

In [5]:
# Get manually inputted model results. [11/25/19 final experiments]

uploaded = files.upload()
model_results2 = pd.read_csv(io.BytesIO(uploaded['model_results2.csv']))

Saving model_results2.csv to model_results2.csv


In [6]:
  !gsutil cp model_results2.csv gs://fb-congressional-data/models/model_results2.csv


Copying file://model_results2.csv [Content-Type=text/csv]...
/ [1 files][  4.2 KiB/  4.2 KiB]                                                
Operation completed over 1 objects/4.2 KiB.                                      


Start here to load new results from bucket.

In [0]:
model_results2 = pd.read_csv("gs://fb-congressional-data/models/model_results2.csv")


In [0]:
# Input metric weights from 0 to 1.
metric_dict2 = {'delta_gender':.1,
                'delta_party':.05,
                'final_gender':.7,
                'final_party':.6,
                'best_gender':.1,
                'best_party':.05,
                'percent_W':.1,
                'percent_D':.05,
                'rocauc_gender':.9,
                'rocauc_party':.8,
                'precision_gender':.5,
                'precision_party':.4,
                'recall_gender':.8,
                'recall_party':.7,
                'pr_auc_gender':1,
                'pr_auc_party':.9
               }

In [0]:
def get_model_metric2(results_df, metric_dict,n):
  '''Take a dataframe of model results, metrics and number of top models
  and return that dataframe with a new model_metric column.'''

  gender_results = results_df[results_df['saved_model_name'].str.contains("gender")]
  party_results = results_df[results_df['saved_model_name'].str.contains("party")]

  gender_best_delta = gender_results.sort_values('delta_val_acc', ascending=False).head(n)
  gender_best_delta['model_metric'] = metric_dict['delta_gender']
  gender_best_final = gender_results.sort_values("final_val_acc", ascending=False).head(n)
  gender_best_final['model_metric'] = metric_dict['final_gender']
  gender_best_acc = gender_results.sort_values("best_val_acc", ascending=False).head(n)
  gender_best_acc['model_metric'] = metric_dict['best_gender']
  gender_best_W = gender_results.sort_values("percent_W", ascending=False).head(n)
  gender_best_W['model_metric'] = metric_dict['percent_W']
  gender_best_rocauc = gender_results.sort_values("rocauc", ascending=False).head(n)
  gender_best_rocauc['model_metric'] = metric_dict['rocauc_gender']
  gender_best_prec = gender_results.sort_values("precision", ascending=False).head(n)
  gender_best_prec['model_metric'] = metric_dict['precision_gender']
  gender_best_recall = gender_results.sort_values("recall", ascending=False).head(n)
  gender_best_recall['model_metric'] = metric_dict['recall_gender']
  gender_best_prauc = gender_results.sort_values("pr_auc", ascending=False).head(n)
  gender_best_prauc['model_metric'] = metric_dict['pr_auc_gender']

  party_best_delta = party_results.sort_values('delta_val_acc', ascending=False).head(n)
  party_best_delta['model_metric'] = metric_dict['delta_party']
  party_best_final = party_results.sort_values("final_val_acc", ascending=False).head(n)
  party_best_final['model_metric'] = metric_dict['final_party']
  party_best_acc = party_results.sort_values("best_val_acc", ascending=False).head(n)
  party_best_acc['model_metric'] = metric_dict['best_party']
  party_best_D = party_results.sort_values("percent_D", ascending=False).head(n)
  party_best_D['model_metric'] = metric_dict['percent_D']
  party_best_rocauc = party_results.sort_values("rocauc", ascending=False).head(n)
  party_best_rocauc['model_metric'] = metric_dict['rocauc_party']
  party_best_prec = party_results.sort_values("precision", ascending=False).head(n)
  party_best_prec['model_metric'] = metric_dict['precision_party']
  party_best_recall = party_results.sort_values("recall", ascending=False).head(n)
  party_best_recall['model_metric'] = metric_dict['recall_party']
  party_best_prauc = party_results.sort_values("pr_auc", ascending=False).head(n)
  party_best_prauc['model_metric'] = metric_dict['pr_auc_party']

  metrics_df = gender_best_delta.append([gender_best_final,gender_best_acc,gender_best_W,gender_best_rocauc,
                                         gender_best_prec,gender_best_recall,gender_best_prauc,party_best_delta,
                                         party_best_delta,party_best_final,party_best_acc,party_best_D,
                                         party_best_rocauc,party_best_prec,party_best_recall,party_best_prauc])
  best_indices = metrics_df.groupby('model_idx')['model_metric'].sum().reset_index()
  best_results_df = results_df.merge(best_indices,on='model_idx')
  return best_results_df

In [10]:
best_models2 = get_model_metric2(model_results2,metric_dict2,3).sort_values('model_metric', ascending=False)
best_models2

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,best_delta_val_acc,percent_W,percent_D,rocauc,precision,recall,pr_auc,saved_model_name,architecture,Unnamed: 40,model_metric
8,Y,2,2625817.0,951162.0,NaN,NaN,1731288.0,310657.0,NaN,NaN,50,30,1000,50,5000,2.0,128,2,0.5,10.0,50,65,0.4975,0.7773,0.8448,0.4676,0.7898,0.8490,0.8493,24,0.0042,0.0045,1.7800,NaN,0.6708,0.5328,0.062300,0.2868,LP_Sun11_24_byo1_model_0_gender_1574661343.7217174.h5,BYO,1.0,5.4
9,Y,2,NaN,NaN,2234496.0,1327042.0,NaN,NaN,1517593.0,524352.0,50,30,1000,50,5000,2.0,128,2,0.5,10.0,50,65,0.5907,0.6784,0.7289,0.5344,0.7150,0.7494,0.7552,7,0.0205,0.0263,NaN,2.99400,0.7023,0.6032,0.070330,0.4455,LP_Sun11_24_byo1_model_0_party_1574661343.7217174.h5,BYO,1.0,5.4
4,Y,1,2752041.0,1003906.0,NaN,NaN,1821965.0,328827.0,NaN,NaN,50,20,1000,20,5000,NaN,128,"[3,4,5]",0.2,NaN,50,63,1.2705,0.7624,0.8504,1.2449,0.7737,0.8486,0.8505,7,-0.0018,0.0001,1.3210,NaN,0.6452,0.5548,0.047950,0.2714,LP_Sun11_24_kim_model_1_gender_1574656083.689345.h5,kim,NaN,5.2
5,Y,1,NaN,NaN,2337054.0,1402563.0,NaN,NaN,1593280.0,557512.0,50,20,1000,20,5000,NaN,128,"[3,4,5]",0.2,NaN,50,63,1.6989,0.6628,0.7285,1.6444,0.6950,0.7501,0.7503,15,0.0216,0.0218,NaN,7.04400,0.6935,0.5659,0.153800,0.4381,LP_Sun11_24_kim_model_1_party_1574656083.689345.h5,kim,NaN,5.2
6,Y,2,2752041.0,1003906.0,NaN,NaN,1821965.0,328827.0,NaN,NaN,50,30,1000,20,5000,1.0,128,2,0.5,10.0,50,64,0.5108,0.7701,0.8488,0.4784,0.7840,0.8482,0.8515,2,-0.0006,0.0027,1.9650,NaN,0.6701,0.5282,0.067880,0.2849,LP_Sun11_24_byo2_model_0_gender_1574663958.4326417.h5,BYO,2.0,4.7
7,Y,2,NaN,NaN,2337054.0,1402563.0,NaN,NaN,1593280.0,557512.0,50,30,1000,20,5000,1.0,128,2,0.5,10.0,50,64,0.6016,0.6684,0.7014,0.5486,0.7035,0.7422,0.7528,28,0.0408,0.0514,NaN,11.38400,0.6836,0.5064,0.222400,0.4241,LP_Sun11_24_byo2_model_0_party_1574663958.4326417.h5,BYO,2.0,4.7
0,Y,1,2752041.0,1003906.0,NaN,NaN,1821965.0,328827.0,NaN,NaN,50,20,1000,40,5000,NaN,128,3,0.2,NaN,50,61,0.4949,0.7772,0.8466,0.4599,0.7910,0.8384,0.8466,1,-0.0082,0.0000,4.5470,NaN,0.6542,0.4045,0.120300,0.2758,LP_Sun11_24_johnson_model_0_gender_1574657050.1533873.h5,johnson,NaN,4.1
1,Y,1,NaN,NaN,2337054.0,1402563.0,NaN,NaN,1593280.0,557512.0,50,20,1000,40,5000,NaN,128,3,0.2,NaN,50,61,0.5726,0.6873,0.7437,0.5161,0.7245,0.7398,0.7489,8,-0.0039,0.0052,NaN,19.18300,0.7058,0.4974,0.368100,0.4603,LP_Sun11_24_johnson_model_0_party_1574657050.1533873.h5,johnson,NaN,4.1
12,Y,2,1965122.0,687034.0,NaN,NaN,1731288.0,310657.0,NaN,NaN,25,30,1000,50,5000,2.0,32,2,0.2,10.0,50,67,0.5053,0.7768,0.8450,0.4689,0.7920,0.8485,0.8486,17,0.0035,0.0036,1.8110,NaN,0.6591,0.5170,0.061530,0.2758,Monday11_25_BYO4_idx20_model_0_gender_1574702721.1645832.h5,BYO,4.0,3.5
13,Y,2,NaN,NaN,1650634.0,990518.0,NaN,NaN,1517593.0,524352.0,25,30,1000,50,5000,2.0,32,2,0.2,10.0,50,67,0.6080,0.6636,0.6830,0.5512,0.7022,0.7480,0.7498,21,0.0650,0.0668,NaN,9.76700,0.6958,0.5244,0.199400,0.4224,Monday11_25_BYO4_idx20_model_0_party_1574702721.1645832.h5,BYO,4.0,3.5


In [11]:
best_models2.head()

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,best_delta_val_acc,percent_W,percent_D,rocauc,precision,recall,pr_auc,saved_model_name,architecture,Unnamed: 40,model_metric
8,Y,2,2625817.0,951162.0,NaN,NaN,1731288.0,310657.0,NaN,NaN,50,30,1000,50,5000,2.0,128,2,0.5,10.0,50,65,0.4975,0.7773,0.8448,0.4676,0.7898,0.8490,0.8493,24,0.0042,0.0045,1.780,NaN,0.6708,0.5328,0.06230,0.2868,LP_Sun11_24_byo1_model_0_gender_1574661343.7217174.h5,BYO,1.0,5.4
9,Y,2,NaN,NaN,2234496.0,1327042.0,NaN,NaN,1517593.0,524352.0,50,30,1000,50,5000,2.0,128,2,0.5,10.0,50,65,0.5907,0.6784,0.7289,0.5344,0.7150,0.7494,0.7552,7,0.0205,0.0263,NaN,2.994,0.7023,0.6032,0.07033,0.4455,LP_Sun11_24_byo1_model_0_party_1574661343.7217174.h5,BYO,1.0,5.4
4,Y,1,2752041.0,1003906.0,NaN,NaN,1821965.0,328827.0,NaN,NaN,50,20,1000,20,5000,NaN,128,"[3,4,5]",0.2,NaN,50,63,1.2705,0.7624,0.8504,1.2449,0.7737,0.8486,0.8505,7,-0.0018,0.0001,1.321,NaN,0.6452,0.5548,0.04795,0.2714,LP_Sun11_24_kim_model_1_gender_1574656083.689345.h5,kim,NaN,5.2
5,Y,1,NaN,NaN,2337054.0,1402563.0,NaN,NaN,1593280.0,557512.0,50,20,1000,20,5000,NaN,128,"[3,4,5]",0.2,NaN,50,63,1.6989,0.6628,0.7285,1.6444,0.6950,0.7501,0.7503,15,0.0216,0.0218,NaN,7.044,0.6935,0.5659,0.15380,0.4381,LP_Sun11_24_kim_model_1_party_1574656083.689345.h5,kim,NaN,5.2
6,Y,2,2752041.0,1003906.0,NaN,NaN,1821965.0,328827.0,NaN,NaN,50,30,1000,20,5000,1.0,128,2,0.5,10.0,50,64,0.5108,0.7701,0.8488,0.4784,0.7840,0.8482,0.8515,2,-0.0006,0.0027,1.965,NaN,0.6701,0.5282,0.06788,0.2849,LP_Sun11_24_byo2_model_0_gender_1574663958.4326417.h5,BYO,2.0,4.7
